<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Hodographs</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="https://unidata.github.io/MetPy/latest/_images/sphx_glr_Advanced_Sounding_001.png" alt="Example Skew-T" style="height: 500px;"></div>

### Questions
1. What is a hodograph?
1. How can MetPy plot hodographs?
1. How can the style of the hodographs be modified to encode other information?

### Objectives
1. <a href="#upperairdata">Obtain upper air data</a>
1. <a href="#simpleplot">Make a simple hodograph</a>
1. <a href="#annotate">Annotate the hodograph with wind vectors</a>
1. <a href="#continuous">Color the plot (continuous)</a>
1. <a href="#segmented">Color the plot (segmented)</a>

<a name="upperairdata"></a>
## Obtain upper air data

Just as we learned in the siphon basics and upper air and skew-T notebook, we need to obtain upperair data to plot. We are going to stick with September 10, 2017 at 00Z for Key West, Fl. If you need a review on obtaining upper air data, please review those lessons.

In [ ]:
from datetime import datetime

from metpy.units import pandas_dataframe_to_unit_arrays
from siphon.simplewebservice.wyoming import WyomingUpperAir

df = WyomingUpperAir.request_data(datetime(1998, 10, 4, 0), 'OUN')
data = pandas_dataframe_to_unit_arrays(df)

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="simpleplot"></a>
## Make a Simple Hodograph

The hodograph is a plot of the wind shear in the sounding. It is constructed by drawing the winds as vectors from the origin and connecting the heads of those vectors. MetPy makes this simple!

In [ ]:
import matplotlib.pyplot as plt
from metpy.plots import Hodograph
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)

h = Hodograph(ax, component_range=60.)
h.add_grid(increment=20)
h.plot(data['u_wind'], data['v_wind'], color='tab:red')

It's relatively common to not want or need to display the entire sounding on a hodograph. Let's limit these data to the lowest 10km and plot it again.

In [ ]:
import metpy.calc as mpcalc
from metpy.units import units
_, u_trimmed, v_trimmed, speed_trimmed, height_trimmed = mpcalc.get_layer(data['pressure'], data['u_wind'],
                                                          data['v_wind'], data['speed'], data['height'],
                                                          heights=data['height'], depth=10 * units.km)

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)

h = Hodograph(ax, component_range=30.)
h.add_grid(increment=10)
h.plot(u_trimmed, v_trimmed, color='tab:red')

<a name="annotate"></a>
## Annotate the hodograph with wind vectors

It may be useful when introducing hodographs to actually show the wind vectors on the plot. The `wind_vectors` method does exactly this. It is often necessary to decimate the wind vectors for the plot to be intelligible.

In [ ]:
h.wind_vectors(u_trimmed[::3], v_trimmed[::3])
fig

We can also set the limits to be asymmetric to beter utilize the plot space.

In [ ]:
ax.set_xlim(-10, 30)
ax.set_ylim(-10, 20)
fig

<a name="continuous"></a>
## Color the plot (continuous)

We can color the line on the hodograph by another variable as well. In the simplest case it will be "continuously" colored, changing with the value of the variable such as windspeed.

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)

h = Hodograph(ax, component_range=30.)
h.add_grid(increment=10)
h.plot_colormapped(u_trimmed, v_trimmed, speed_trimmed)

In [ ]:
from metpy.plots import colortables
import numpy as np

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)

norm, cmap = colortables.get_with_range('Carbone42', np.min(speed_trimmed), np.max(speed_trimmed))

h = Hodograph(ax, component_range=30.)
h.add_grid(increment=10)
l = h.plot_colormapped(u_trimmed, v_trimmed, speed_trimmed, cmap=cmap, norm=norm)
plt.colorbar(l, ticks=np.arange(0, 30, 5))

<a name="segmented"></a>
## Color the plot (segmented)

It may be useful when introducing hodographs to actually show the wind vectors on the plot. The `wind_vectors` method does exactly this. It is often necessary to decimate the wind vectors for the plot to be intelligible.

We can also color the hodograph based on another variable - either continuously or in a segmented way. Here we'll color the hodograph by height above ground level.

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)

boundaries = np.array([0, 1, 3, 5, 8]) * units.km
colors = ['tab:red', 'tab:green', 'tab:blue', 'tab:olive']

# Since we want to do things in terms of AGL, we need to make AGL heights
agl = height_trimmed - height_trimmed[0]

h = Hodograph(ax, component_range=30.)
h.add_grid(increment=10)
l = h.plot_colormapped(u_trimmed, v_trimmed, agl, bounds=boundaries, colors=colors)
plt.colorbar(l)

<a href="#top">Top</a>
<hr style="height:2px;">